 # <center>Segmenting and Clustering Neighborhoods in Toronto </center>
<img src="https://www.toronto.ca/wp-content/uploads/2017/12/9578-moving-to-toronto-7.png" width=90% align="center"/>

In [12]:
# First we have to install beutifulsoup for scraping the website
#!conda install -c anaconda beautifulsoup4 --yes

In [13]:
# Install html parser (lxml or html5lib)
#!conda install -c anaconda lxml --yes

In [3]:
# imports
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [4]:
# Assign the source (Wikipedia)
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')

# locate the table 
table = soup.find('table', class_='wikitable')

headers=list()
# retrieve the headers
for row in table.findAll("th"):
    headers.append(row.text.replace('\n',''))

#Create the dataframe
df=pd.DataFrame(columns=headers)

# append the data
rows=list()
for tbody in table.findAll("tbody"):
    for tr in tbody.findAll("tr"):
        for td in tr.findAll("td"):
            rows.append(td.text.replace('\n',''))
        
        if len(rows)>0:
            df = df.append({headers[0]: rows[0], headers[1]: rows[1], headers[2]: rows[2]}, ignore_index=True, sort=False)
        rows.clear()

# check the df        
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [5]:
# Group by aggregating the neighbourhoods 
aggregations = {
    'Neighbourhood': lambda x:', '.join(x)
}
df = df.groupby(['Postcode', 'Borough']).agg(aggregations)
# reset index
df = df.reset_index()

df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M1B,Scarborough,"Rouge, Malvern"
2,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn


In [6]:
# Delete not assigned boroughs
df.drop(df.loc[df['Borough']=='Not assigned'].index, inplace=True)
df = df.sort_values(by=['Postcode'])

df.head()

,Postcode,Borough,Neighbourhood
1,M1B,Scarborough,"Rouge, Malvern"
2,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn
5,M1H,Scarborough,Cedarbrae


In [7]:
#Let's check if there are some not assigned values in neighbourhood column
df.loc[df['Neighbourhood'] == 'Not assigned']

,Postcode,Borough,Neighbourhood
120,M7A,Queen's Park,Not assigned


In [8]:
# Copy values from borough column to those not assigned values on neighbourhood column
df.Neighbourhood = df.Borough.where(df.Neighbourhood == 'Not assigned', df.Neighbourhood)

In [9]:
# Let's check again if there are some not assigned values in neighbourhood column
df.loc[df['Neighbourhood'] == 'Not assigned']

,Postcode,Borough,Neighbourhood


In [13]:
df.head()

,Postcode,Borough,Neighbourhood
1,M1B,Scarborough,"Rouge, Malvern"
2,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn
5,M1H,Scarborough,Cedarbrae


In [14]:
df.shape

(103, 3)